In [1]:
from pathlib import Path

# Remote Sensing DDPM
from remote_sensing_ddpm.train_downstream_tasks import create_wandb_run_name

# Lit-Diffusion
from lit_diffusion.diffusion_base.constants import LOGGING_TRAIN_PREFIX, LOGGING_VAL_PREFIX
from lit_diffusion.constants import (
    PL_WANDB_LOGGER_CONFIG_KEY,
)

# FWTP
from from_wandb_to_paper.data.wandb_data import get_wandb_run_histories, convert_run_histories_to_df
from from_wandb_to_paper.data.constants import Resolution

In [2]:
FEATURE_EXTRACTOR_FILES = Path("../../config/model_configs/downstream_tasks/feature_extractors")
EWC_CLASSIFICATION_CONFIG_PATH = Path("../../config/model_configs/downstream_tasks/ewc-segmentation.yaml.yaml")
EXPERIMENT_NAMES = [
    create_wandb_run_name(backbone_name=backbone_path.name, downstream_head_name=EWC_CLASSIFICATION_CONFIG_PATH.name)
    for backbone_path in FEATURE_EXTRACTOR_FILES.glob("*.yaml")
]
WAND_PROJECT_ID = "ssl-diffusion/rs-ddpm-ms-segmentation"
CLASSIFICATION_METRICS = ["accuracy", "mIoU"]

RUN_FILTER = {f"display_name": {"$in": ["s2_s1-ewc-segmentation"]}}

In [3]:
import wandb
from tqdm import tqdm
run_results = {}
api = wandb.Api(timeout=15)
runs = [api.run(f"{WAND_PROJECT_ID}/{run_id}") for run_id in ["ulhi2udw", "crzlqmax"]]

for run in runs:
    single_run_complete_history = []
    # Get all data
    for x in tqdm(run.scan_history(), desc="Loading history"):
        single_run_complete_history.append(x)
    run_results[run.id] = {
        "id": run.id,
        "name": run.name,
        "history": single_run_complete_history,
    }

Loading history: 27680it [00:17, 1578.14it/s]
Loading history: 32780it [00:18, 1731.78it/s]


In [7]:
ulhi2ud_hist = run_results["ulhi2udw"]["history"]
ulhi2udw_GS = [ulhi2ud_hist[i]["trainer/global_step"] for i in range(len(ulhi2ud_hist))]
ulhi2udw_epoch = [ulhi2ud_hist[i]["epoch"] for i in range(len(ulhi2ud_hist))]

In [13]:
crzlqmax_hist = run_results["crzlqmax"]["history"]
crzlqmax_GS = [crzlqmax_hist[i]["trainer/global_step"] for i in range(len(crzlqmax_hist))]
crzlqmax_epoch = [crzlqmax_hist[i]["epoch"] for i in range(len(crzlqmax_hist))]

In [11]:
len(ulhi2udw_epoch)

27680

In [14]:
len(crzlqmax_epoch)

32780

In [23]:
import pandas as pd
crzlqmax_df = pd.DataFrame({"epoch": crzlqmax_epoch, "trainer/global_step": crzlqmax_GS})
ulhi2udw_df = pd.DataFrame({"epoch": ulhi2udw_epoch, "trainer/global_step": ulhi2udw_GS})

In [34]:
crzlqmax_df = crzlqmax_df.dropna()
crzlqmax_df[(crzlqmax_df["epoch"] - crzlqmax_df["epoch"].shift(1)).fillna(0).astype(bool)]

,epoch,trainer/global_step
7768,1.0,3882
15534,2.0,7764
23300,3.0,11646
31066,4.0,15528


In [35]:
ulhi2udw_df = ulhi2udw_df.dropna()
ulhi2udw_df[(ulhi2udw_df["epoch"] - ulhi2udw_df["epoch"].shift(1)).fillna(0).astype(bool)]

,epoch,trainer/global_step
8254,1.0,3882
16506,2.0,7764
24758,3.0,11646


In [36]:
run_results["crzlqmax"]["name"], run_results["ulhi2udw"]["name"]

('s2_s1-ewc-segmentation-no-val', 's2_s1-ewc-segmentation')